In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!kaggle datasets download -d muhammadshahidazeem/customer-churn-dataset

  0% 0.00/6.66M [00:00<?, ?B/s]
100% 6.66M/6.66M [00:00<00:00, 88.9MB/s]


In [ ]:
!unzip /content/customer-churn-dataset.zip

Archive:  /content/customer-churn-dataset.zip
  inflating: customer_churn_dataset-testing-master.csv  
  inflating: customer_churn_dataset-training-master.csv  


In [ ]:
import pandas as pd
import numpy as np
import tensorflow  as tf

In [ ]:
df = pd.read_csv("/content/customer_churn_dataset-training-master.csv")
df

,CustomerID,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Subscription Type,Contract Length,Total Spend,Last Interaction,Churn
0,2.0,30.0,Female,39.0,14.0,5.0,18.0,Standard,Annual,932.00,17.0,1.0
1,3.0,65.0,Female,49.0,1.0,10.0,8.0,Basic,Monthly,557.00,6.0,1.0
2,4.0,55.0,Female,14.0,4.0,6.0,18.0,Basic,Quarterly,185.00,3.0,1.0
3,5.0,58.0,Male,38.0,21.0,7.0,7.0,Standard,Monthly,396.00,29.0,1.0
4,6.0,23.0,Male,32.0,20.0,5.0,8.0,Basic,Monthly,617.00,20.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
440828,449995.0,42.0,Male,54.0,15.0,1.0,3.0,Premium,Annual,716.38,8.0,0.0
440829,449996.0,25.0,Female,8.0,13.0,1.0,20.0,Premium,Annual,745.38,2.0,0.0
440830,449997.0,26.0,Male,35.0,27.0,1.0,5.0,Standard,Quarterly,977.31,9.0,0.0
440831,449998.0,28.0,Male,55.0,14.0,2.0,0.0,Standard,Quarterly,602.55,2.0,0.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 440833 entries, 0 to 440832
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   CustomerID         440832 non-null  float64
 1   Age                440832 non-null  float64
 2   Gender             440832 non-null  object 
 3   Tenure             440832 non-null  float64
 4   Usage Frequency    440832 non-null  float64
 5   Support Calls      440832 non-null  float64
 6   Payment Delay      440832 non-null  float64
 7   Subscription Type  440832 non-null  object 
 8   Contract Length    440832 non-null  object 
 9   Total Spend        440832 non-null  float64
 10  Last Interaction   440832 non-null  float64
 11  Churn              440832 non-null  float64
dtypes: float64(9), object(3)
memory usage: 40.4+ MB


In [ ]:
df.isnull().sum()

CustomerID           1
Age                  1
Gender               1
Tenure               1
Usage Frequency      1
Support Calls        1
Payment Delay        1
Subscription Type    1
Contract Length      1
Total Spend          1
Last Interaction     1
Churn                1
dtype: int64

In [ ]:
for column in df:
    if df[column].dtypes=='object':
        print(f'{column}: {df[column].unique()}')

Gender: ['Female' 'Male' nan]
Subscription Type: ['Standard' 'Basic' 'Premium' nan]
Contract Length: ['Annual' 'Monthly' 'Quarterly' nan]


Label Encoder && One Hot Encoding

In [ ]:
X = df.iloc[:, 1:-1].values
y = df.iloc[:, -1].values

In [ ]:
print(X)

[[30.0 'Female' 39.0 ... 'Annual' 932.0 17.0]
 [65.0 'Female' 49.0 ... 'Monthly' 557.0 6.0]
 [55.0 'Female' 14.0 ... 'Quarterly' 185.0 3.0]
 ...
 [26.0 'Male' 35.0 ... 'Quarterly' 977.31 9.0]
 [28.0 'Male' 55.0 ... 'Quarterly' 602.55 2.0]
 [31.0 'Male' 48.0 ... 'Quarterly' 567.77 21.0]]


In [ ]:
index = df.columns.get_loc('Contract Length')
print(index)

8


In [ ]:
df2 = pd.get_dummies(data=df , columns=['Subscription Type', 'Contract Length'])
df2 = df2.reindex(columns=['Age','Gender','Tenure','Usage Frequency','Support Calls','Payment Delay','Total Spend','Last Interaction','Subscription Type_Basic','Subscription Type_Premium','Subscription Type_Standard','Contract Length_Annual','Contract Length_Monthly','Contract Length_Quarterly','Churn'])
df2

,Age,Gender,Tenure,Usage Frequency,Support Calls,Payment Delay,Total Spend,Last Interaction,Subscription Type_Basic,Subscription Type_Premium,Subscription Type_Standard,Contract Length_Annual,Contract Length_Monthly,Contract Length_Quarterly,Churn
0,30.0,Female,39.0,14.0,5.0,18.0,932.00,17.0,0,0,1,1,0,0,1.0
1,65.0,Female,49.0,1.0,10.0,8.0,557.00,6.0,1,0,0,0,1,0,1.0
2,55.0,Female,14.0,4.0,6.0,18.0,185.00,3.0,1,0,0,0,0,1,1.0
3,58.0,Male,38.0,21.0,7.0,7.0,396.00,29.0,0,0,1,0,1,0,1.0
4,23.0,Male,32.0,20.0,5.0,8.0,617.00,20.0,1,0,0,0,1,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
440828,42.0,Male,54.0,15.0,1.0,3.0,716.38,8.0,0,1,0,1,0,0,0.0
440829,25.0,Female,8.0,13.0,1.0,20.0,745.38,2.0,0,1,0,1,0,0,0.0
440830,26.0,Male,35.0,27.0,1.0,5.0,977.31,9.0,0,0,1,0,0,1,0.0
440831,28.0,Male,55.0,14.0,2.0,0.0,602.55,2.0,0,0,1,0,0,1,0.0


In [ ]:
X = df2.iloc[:, 0:-1].values
y = df2['Churn']
print(X)

[[30.0 'Female' 39.0 ... 1 0 0]
 [65.0 'Female' 49.0 ... 0 1 0]
 [55.0 'Female' 14.0 ... 0 0 1]
 ...
 [26.0 'Male' 35.0 ... 0 0 1]
 [28.0 'Male' 55.0 ... 0 0 1]
 [31.0 'Male' 48.0 ... 0 0 1]]


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 1] = le.fit_transform(X[:, 1])
print(X)

[[30.0 0 39.0 ... 1 0 0]
 [65.0 0 49.0 ... 0 1 0]
 [55.0 0 14.0 ... 0 0 1]
 ...
 [26.0 1 35.0 ... 0 0 1]
 [28.0 1 55.0 ... 0 0 1]
 [31.0 1 48.0 ... 0 0 1]]


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, classification_report
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

In [ ]:
X_train.shape

(308583, 14)

In [ ]:
import keras
model = keras.Sequential([
    keras.layers.Dense(3000, input_shape=(14,), activation='relu'),
    keras.layers.Dense(2500, activation='relu'),
    keras.layers.Dense(1000, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs = 5)

Epoch 1/5
9644/9644 [==============================] - 1242s 129ms/step - loss: 0.0682 - accuracy: 0.9762
Epoch 2/5
9644/9644 [==============================] - 1215s 126ms/step - loss: 0.0410 - accuracy: 0.9865
Epoch 3/5
9644/9644 [==============================] - 1214s 126ms/step - loss: 0.0336 - accuracy: 0.9889
Epoch 4/5
9644/9644 [==============================] - 1220s 127ms/step - loss: 0.0300 - accuracy: 0.9901
Epoch 5/5
9644/9644 [==============================] - 1215s 126ms/step - loss: 0.0273 - accuracy: 0.9908


In [ ]:
model.evaluate(X_test, y_test)

4133/4133 [==============================] - 95s 23ms/step - loss: nan - accuracy: 0.9468


[nan, 0.9467977285385132]

In [ ]:
y_predict = model.predict(X_test)

4133/4133 [==============================] - 89s 21ms/step


In [ ]:
print(model.predict(scaler.transform([[42,1,54,15,1,3,716,8,0,1,0,1,0,0]]))> 0.5)

1/1 [==============================] - 0s 31ms/step
[[False]]
